In [1]:
import glob
import json
import os
from pathlib import Path
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

In [2]:
def get_library_size(end_library):
    if end_library is None:
        return 0
    else:
        cnt = 0
        for l in end_library:
            if l is not None:
                cnt += 1
    return cnt
    
def get_expressions_data(dir, globstr, savename=None):
    data = []
    for f in dir.glob(globstr):
        run = int(re.search("run_(\d+)\.", f.name).group(1))
        restart = int(re.search("run_\d+\.(\d+)", f.name).group(1))
        dataset = re.search("vsize\.([a-zA-Z]+)\.", f.name).group(1)
        d = json.load(f.open("r"))
        results = d["results"]
        for res in results:
            task_name = re.search('name=(.*),', res["task"]).group(1)
            data.append((run, task_name, restart, dataset, res["success"], res["num_values_explored"]))
        df = pd.DataFrame.from_records(data, columns=["Run", "TaskName", "Restart", "DataSet", "Success", "NumValuesExplored",])
        if savename:
            df.to_pickle(savename)
    return df

In [3]:
%cd "/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results/"
globstr = "run*.json"
dir = Path("/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results/")
data = get_expressions_data(dir, globstr)

/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results


In [4]:
data

,Run,TaskName,Restart,DataSet,Success,NumValuesExplored
0,3,addDecimal,2,new,False,25024
1,3,addDecimal,2,new,True,608
2,3,addPlusSign,2,new,True,351
3,3,appendAmOrPm,2,new,False,25072
4,3,appendAmOrPm,2,new,False,25027
...,...,...,...,...,...,...
2531,2,truncateIfTooLong,8,new,False,6341
2532,2,truncateIfTooLong,8,new,False,6259
2533,2,twoLetterAcronym,8,new,True,3312
2534,2,twoLetterAcronymCapitalization,8,new,False,6337


In [66]:
import pandas as pd

result_processed = {}
for restart in [2, 4, 8]:
    if restart not in result_processed:
        result_processed[restart] = {}
    for dataset in ["sygus", "new"]:
        temp_data = []
        df = data[(data["Restart"] == restart) & (data["DataSet"] == dataset) & (data["Success"] == True)]
        for i in range(1, 50102, 100):
            filtered_df = df[df["NumValuesExplored"] <= i].groupby("Run").count()
            # Ensure the DataFrame is not empty
            if not filtered_df.empty:
                mean = filtered_df.mean()[0]
                std = filtered_df.std()[0]
                temp_data.append({"evaluations": i, "avg": mean, "std": std})
            else:
                temp_data.append({"evaluations": i, "avg": 0, "std": 0})  # Or handle this case as you see fit

        result_processed[restart][dataset] = pd.DataFrame(temp_data)
        
        path = "/Users/habib/Documents/ags-bee-search/scripts/result_processor_crossbeam/stopping_fix/eval_data"

        file_name = f'{dataset}-50k_{restart}restart_grow.csv'
        file_path = os.path.join(path, file_name)
        result_processed[restart][dataset].to_csv(file_path, index=False)
                        


In [55]:
print(data_processd[2]["sygus"].head())

     Run      TaskName  Growth DataSet  Success  NumValuesExplored
393    2      bikes.sl       2   sygus     True               1091
394    2    dr-name.sl       2   sygus     True               1712
395    2  firstname.sl       2   sygus     True                410
396    2   initials.sl       2   sygus     True               4472
397    2   lastname.sl       2   sygus     True                336


In [32]:
data_sygus['Success'] = data_sygus['Success'].astype(int)
print(data_sygus.shape)

# # Define bins for 'NumValuesExplored'
# max_value = data_sygus['NumValuesExplored'].max()
# bins = range(0, max_value + 1000)  # Adjust the step and range as needed

# # Assign each row to a bin
# data_sygus['ExploredBin'] = pd.cut(data_sygus['NumValuesExplored'], bins=bins, labels=bins[:-1], right=False)

# # Group by bins and calculate the sum of 'Success'
# grouped = data_sygus.groupby('ExploredBin')['Success'].sum()

# # Plot
# grouped.plot(kind='bar')
# plt.xlabel('NumValuesExplored (Range)')
# plt.ylabel('Number of Problems Solved')
# plt.title('Number of Problems Solved vs. NumValuesExplored Range')
# plt.xticks(rotation=45)
# plt.show()

(1733, 5)


/var/folders/42/dmtfwt_d4tzcz1_nf6n174ww0000gn/T/ipykernel_89455/2994594252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sygus['Success'] = data_sygus['Success'].astype(int)


In [ ]:
%cd "/Users/habib/Documents/ags-bee-search/src_a_crossbeam/augmented/bustle_results/"
d = pickle.load(Path("sygus-50k_grow-aggregated.pkl").open("rb"))

In [53]:
d

,Run,TaskName,Success,NumValuesExplored
0,1,11604909.sl,1,24518
1,1,17212077.sl,1,841
2,1,2171308.sl,1,2742
3,1,25239569.sl,0,50025
4,1,28627624_1.sl,1,797
...,...,...,...,...
440,5,univ_2.sl,1,41549
441,5,univ_3.sl,0,50093
442,5,univ_4.sl,0,50090
443,5,univ_5.sl,0,50031


In [11]:
d1 = data[data["Run"] == 1]
d1succ = d1[d1["Success"] == 1]
means = []
stds = []
dsucc = data[data["Success"] == 1].drop(columns=["TaskName"])
#for x in range(1, 50001, 1):
s = dsucc[dsucc["NumValuesExplored"] <= 50000].groupby(by=["Run"]).sum()
print(s["Success"].mean())
print(s["Success"].std())
    

304.2
2.6832815729997477


In [85]:
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup"
globstr = "run*.json"
dirs = ["50k_nogrow", "50k_2restart_grow", "50k_2restart_nogrow", "50k_4restart_grow", "50k_4restart_nogrow", "50k_8restart_grow", "50k_8restart_nogrow"]
inc = 100
valuations = list(range(1, 50000+inc, inc))
for dir in dirs:
    df = get_expressions_data(Path(dir), globstr, f"{dir}.pkl")
    newdf = df[df["DataSet"] == "new"].drop(columns=["DataSet"])
    sygusdf = df[df["DataSet"] == "sygus"].drop(columns=["DataSet"])
    newdf = newdf.groupby(by=["Run", "TaskName"], as_index=False).sum()
    sygusdf = sygusdf.groupby(by=["Run", "TaskName"], as_index=False).sum()
    newdf.to_pickle(f"new-{dir}-aggregated.pkl")
    sygusdf.to_pickle(f"sygus-{dir}-aggregated.pkl")

    for df, name in [(newdf, f"new-{dir}.csv"), (sygusdf, f"sygus-{dir}.csv")]:
        means = []
        stds = []
        succ = df[df["Success"] == 1].drop(columns=["TaskName"])
        succ = succ.sort_values(by="NumValuesExplored")
        for val in valuations:
            vsucc = succ[succ["NumValuesExplored"] <= val].groupby(by=["Run"]).sum()
            mean = 0. if vsucc.empty else vsucc["Success"].mean()
            std = 0. if vsucc.empty else vsucc["Success"].std()
            means.append(mean)
            stds.append(std)
        csvdf = pd.DataFrame({"valuations": valuations, "avg": means, "std": stds})
        csvdf.to_csv(name, index=False)

/home/ken/Envs/augmente_bustle_results/bustle_results_nodup


In [13]:
%cd "/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/"
d = pickle.load(Path("50k_4restart_nogrow.pkl").open("rb"))
x = d.groupby(by=["TaskName", "Run"]).sum()
x.loc["11604909.sl"]

[Errno 2] No such file or directory: '/home/ken/Projects/augmented_bustle_results/augmented/bustle_results_nodup/'
/home/ken/Envs/augmente_bustle_results/bustle_results_nodup


,DataSet,Attempt,Success,ElapsedTime,NumUniqueValues,NumValuesExplored,EndLibrarySize
Run,,,,,,,
1,sygus,1,1,8.801042,1375,9795,0
2,sygus,1,1,3.593925,774,4201,0
3,sygussygus,3,1,17.017261,2738,19415,0
4,sygus,1,1,10.156737,1526,11396,0
5,sygus,1,1,2.801516,607,2798,0


In [ ]:
d.gr

In [95]:
def get_tasknames(runname):
    names = set()
    d = json.load(Path(runname).open("r"))
    results = d["results"]
    for res in results:
        task_name = re.search('name=(.*),', res["task"]).group(1)
        names.add(task_name)
    return names
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours/600s"
sygus_tasknames = get_tasknames("run_1.50000.attempt-86400.600.grow-True.vw-bustle_sig-vsize.sygus.json")
new_tasknames = get_tasknames("run_1.50000.attempt-86400.600.grow-True.vw-bustle_sig-vsize.new.json")
print(len(sygus_tasknames))
print(len(new_tasknames))

/home/ken/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours/600s
89
38


In [96]:
def get_partial_time_results(dirs, globstr):
    solved_data = []
    for dir in dirs:
        for f in Path(dir).glob(globstr):
            run = int(re.search("run_(\d+)\.", f.name).group(1))
            d = json.load(f.open("r"))
            results = d["results"]
            for i in range(len(results)):
                if results[i]["success"]:
                    task_name = re.search('name=(.*),', results[i]["task"]).group(1)
                    attempt = results[i]["attempt"]
                    attempts_data = results[i - attempt + 1 : i + 1]
                    explored = sum(r["num_values_explored"] for r in attempts_data)
                    seconds = sum(r["elapsed_time"] for r in attempts_data)
                    library_size = sum(get_library_size(r["ending_library"]) for r in attempts_data)
                    solved_data.append((run, task_name, attempt, True, library_size, explored, seconds))
    df = pd.DataFrame.from_records(solved_data, columns=["Run", "TaskName", "Attempts", "Success", "EndLibrarySize", "NumValuesExplored", "ElapsedTime"])
    return df

In [97]:
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours"
dirs = ["600s"]
partial_sygus_timedata = get_partial_time_results(dirs, "run*sygus.json")
partial_new_timedata = get_partial_time_results(dirs, "run*new.json")

/home/ken/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours


In [98]:
partial_new_timedata

,Run,TaskName,Attempts,Success,EndLibrarySize,NumValuesExplored,ElapsedTime
0,4,addDecimal,1,True,0,18886,22.956334
1,4,addPlusSign,1,True,0,2589,1.878894
2,4,appendAmOrPm,2,True,2,56188,52.853697
3,4,capitalizeCityAndState,1,True,0,2169,1.600643
4,4,capitalizeSentence,1,True,0,2091,1.513061
...,...,...,...,...,...,...,...
182,2,suffixRemainder,1,True,0,1908,1.453009
183,2,swapCase,1,True,0,403,0.258018
184,2,truncateIfTooLong,2,True,2,51951,53.180535
185,2,twoLetterAcronym,1,True,0,27251,23.959625


In [99]:
def get_partial_task_time_results(dirs, globstr):
    data = []
    for d in dirs:
        dir = Path(d)
        for f in Path(dir).glob(globstr):
            run = int(re.search("run_(\d+)\.", f.name).group(1))
            d = json.load(f.open("r"))
            results = d["results"]
            task_name = re.search('name=(.*),', results[0]["task"]).group(1)
            attempts = 0
            explored = 0
            seconds = 0
            library_size = 0
            for i in range(len(results)):
                attempts += 1
                explored += results[i]["num_values_explored"]
                seconds += results[i]["elapsed_time"]
                library_size += get_library_size(results[i]["ending_library"])
                if i == len(results) - 1:
                    success = results[i]["success"]
                    if d == "86400s" or success:
                        data.append((run, task_name, attempts, success, library_size, explored, seconds))
        df = pd.DataFrame.from_records(data, columns=["Run", "TaskName", "Attempts", "Success", "EndLibrarySize", "NumValuesExplored", "ElapsedTime"])
    return df

In [100]:
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours"
dirs = ["3600s", "21600s", "86400s"]
partial_sygus_tasks_timedata = get_partial_task_time_results(dirs, "run*sygus*json")
partial_new_tasks_timedata = get_partial_task_time_results(dirs, "run*new*json")

/home/ken/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours


In [101]:
sygus_timedata = pd.concat([partial_sygus_timedata, partial_sygus_tasks_timedata], ignore_index=True)
new_timedata = pd.concat([partial_new_timedata, partial_new_tasks_timedata], ignore_index=True)

In [102]:
sygus_timedata

,Run,TaskName,Attempts,Success,EndLibrarySize,NumValuesExplored,ElapsedTime
0,3,bikes.sl,1,True,0,964,1.601188
1,3,dr-name.sl,1,True,0,1740,1.275079
2,3,firstname.sl,1,True,0,408,0.293359
3,3,initials.sl,1,True,0,27272,26.481226
4,3,lastname.sl,1,True,0,330,0.242270
...,...,...,...,...,...,...,...
414,2,stackoverflow4.sl,53,True,0,2604989,2349.573812
415,3,stackoverflow4.sl,4,True,0,158892,141.872676
416,4,univ_6.sl,8,True,35,360377,339.400350
417,4,phone-10.sl,158,True,0,7890659,7090.201260


In [103]:
import itertools
def verify_timedata(df, tasknames):
    not_in_df = []
    for run, taskname in itertools.product(range(1,6), tasknames):
        rows = df.loc[(df["TaskName"] == taskname) & (df["Run"] == run)]
        if len(rows) == 0:
            print(run, taskname, "not in DF")
            not_in_df.append((run, taskname))
        elif len(rows) > 1:
            print(run, taskname, "has multiple entries in DF")
    return not_in_df

In [104]:
print("SYGUS")
sygus_manual_entries = verify_timedata(sygus_timedata, sygus_tasknames)
print("NEW")
new_manual_entries = verify_timedata(new_timedata, new_tasknames)

SYGUS
1 univ_5.sl not in DF
1 stackoverflow8.sl not in DF
1 25239569.sl not in DF
1 remove-leading-and-trailing-spaces-from-text.sl not in DF
1 43120683.sl not in DF
2 stackoverflow8.sl not in DF
2 25239569.sl not in DF
2 phone-10.sl not in DF
2 remove-leading-and-trailing-spaces-from-text.sl not in DF
2 univ_6.sl not in DF
2 43120683.sl not in DF
3 stackoverflow8.sl not in DF
3 25239569.sl not in DF
3 phone-10.sl not in DF
3 remove-leading-and-trailing-spaces-from-text.sl not in DF
3 43120683.sl not in DF
4 univ_5.sl not in DF
4 stackoverflow8.sl not in DF
4 25239569.sl not in DF
4 remove-leading-and-trailing-spaces-from-text.sl not in DF
4 43120683.sl not in DF
5 stackoverflow8.sl not in DF
5 25239569.sl not in DF
5 phone-10.sl not in DF
5 remove-leading-and-trailing-spaces-from-text.sl not in DF
5 43120683.sl not in DF
NEW


In [105]:
def get_manual_entries(entries):
    data = []
    for run, tname in entries:
        data.append((run, tname, np.nan, False, np.nan, np.nan, np.nan))
    df = pd.DataFrame.from_records(data, columns=["Run", "TaskName", "Attempts", "Success", "EndLibrarySize", "NumValuesExplored", "ElapsedTime"])
    return df

In [106]:
manual_df = get_manual_entries(sygus_manual_entries)
sygus_time_df = pd.concat([sygus_timedata, manual_df], ignore_index=True)
sygus_time_df["Dataset"] = "sygus"
new_timedata["Dataset"] = "new"
time_data = pd.concat([sygus_time_df, new_timedata], ignore_index=True)
time_data.to_pickle("50k_24hours.pkl")

In [2]:
%cd "~/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours"
d = pickle.load(Path("50k_24hours.pkl").open("rb"))
d

/home/ken/Envs/augmente_bustle_results/bustle_results_nodup/50k_24hours


,Run,TaskName,Attempts,Success,EndLibrarySize,NumValuesExplored,ElapsedTime,Dataset
0,3,bikes.sl,1.0,True,0.0,964.0,1.601188,sygus
1,3,dr-name.sl,1.0,True,0.0,1740.0,1.275079,sygus
2,3,firstname.sl,1.0,True,0.0,408.0,0.293359,sygus
3,3,initials.sl,1.0,True,0.0,27272.0,26.481226,sygus
4,3,lastname.sl,1.0,True,0.0,330.0,0.242270,sygus
...,...,...,...,...,...,...,...,...
630,2,twoLetterAcronym,1.0,True,0.0,27251.0,23.959625,new
631,2,twoLetterAcronymCapitalization,1.0,True,0.0,7961.0,6.695381,new
632,1,replaceAllCompany,11.0,True,65.0,509895.0,481.067420,new
633,5,replaceAllCompany,11.0,True,5.0,543196.0,565.168201,new


In [3]:
sygustime = d[d["Dataset"] == "sygus"].drop(columns=["Dataset"])
newtime = d[d["Dataset"] == "new"].drop(columns=["Dataset"])
sygustime

,Run,TaskName,Attempts,Success,EndLibrarySize,NumValuesExplored,ElapsedTime
0,3,bikes.sl,1.0,True,0.0,964.0,1.601188
1,3,dr-name.sl,1.0,True,0.0,1740.0,1.275079
2,3,firstname.sl,1.0,True,0.0,408.0,0.293359
3,3,initials.sl,1.0,True,0.0,27272.0,26.481226
4,3,lastname.sl,1.0,True,0.0,330.0,0.242270
...,...,...,...,...,...,...,...
440,5,stackoverflow8.sl,NaN,False,NaN,NaN,NaN
441,5,25239569.sl,NaN,False,NaN,NaN,NaN
442,5,phone-10.sl,NaN,False,NaN,NaN,NaN
443,5,remove-leading-and-trailing-spaces-from-text.sl,NaN,False,NaN,NaN,NaN


In [30]:
toadd = [[2, "phone-10.sl"], [2, "remove-leading-and-trailing-spaces-from-text.sl"], [4, "43120683.sl"], [5, "remove-leading-and-trailing-spaces-from-text.sl"]]
updates = [367, 41945, 4722, 32014]

In [44]:
df = sygustime
df.reset_index()
for (run, tname), time in zip(toadd, updates):
    index = df.index[(df["Run"] == run) & (df["TaskName"] == tname)].tolist()[0]
    df.loc[index, ["Success", "ElapsedTime"]] = [True, time]

In [47]:
df[(df["Run"] == 2) & (df["TaskName"] == "phone-10.sl")]

,Run,TaskName,Attempts,Success,EndLibrarySize,NumValuesExplored,ElapsedTime
426,2,phone-10.sl,NaN,True,NaN,NaN,367.0


In [48]:
inc = 1
seconds = list(range(1, 86400+inc, inc))
for df, name in [(df, f"sygus-time-updated.csv")]:#, (sygustime, f"sygus-time.csv")]:#, ]:
        means = []
        stds = []
        succ = df[df["Success"] == 1].drop(columns=["EndLibrarySize", "NumValuesExplored", "Attempts", "TaskName"])
        succ = succ.sort_values(by="ElapsedTime")
        for s in seconds:
            vsucc = succ[succ["ElapsedTime"] <= s].groupby(by=["Run"]).sum()
            mean = 0. if vsucc.empty else vsucc["Success"].mean()
            std = 0. if vsucc.empty else vsucc["Success"].std()
            means.append(mean)
            stds.append(std)
        csvdf = pd.DataFrame({"seconds": seconds, "avg": means, "std": stds})
        csvdf.to_csv(name, index=False)